In [26]:
source('helpers.R')
source('../utils.R')

loadLibraries()
x = loadData()

In [28]:
df.combined =  x$combined %>% 
    mutate(MSSubClass = as.character(MSSubClass), 
           YearBuiltChar = as.character(YearBuilt),
           LotAreaSqrt = sqrt(LotArea),
           LotAreaLog = log(LotFrontage)
    )

df.lot_frontage = df.combined %>% filter(!is.na(LotFrontage))
df.lot_frontage.na = df.combined %>% filter(is.na(LotFrontage))
df.lot_frontage.cutted = df.lot_frontage %>% filter(LotFrontage < 130)

df.train = df.combined %>% filter(dataSource == "train") %>% mutate(LotFrontageCalc = NA)
df.train.copy = df.combined %>% filter(dataSource == "train") %>% mutate(LotFrontageCalc = NA)

In [29]:
df.data.BrkSide = df.lot_frontage %>% filter(Neighborhood == 'BrkSide')
df.data.BrkSide.RL = df.data.BrkSide %>% filter(MSZoning == "RL")
df.data.BrkSide.RM = df.data.BrkSide %>% filter(MSZoning == "RM")

df.train.BrkSide = df.train %>% filter(Neighborhood == 'BrkSide')
df.train.BrkSide.RL = df.train.BrkSide %>% filter(MSZoning == "RL")
df.train.BrkSide.RM = df.train.BrkSide %>% filter(MSZoning == "RM")

lm.BrkSide.RL = lm(LotFrontage ~ LotAreaSqrt, data = df.data.BrkSide.RL)
predicted = predict(lm.BrkSide.RL, df.train.BrkSide.RL)

In [35]:
df.train[df.train$Neighborhood == 'BrkSide' & df.train$MSZoning == 'RL', "LotFrontageCalc"] = predicted

head(df.train %>% filter(Neighborhood == 'BrkSide' & MSZoning == "RL") %>% select(LotFrontageCalc))

LotFrontageCalc
59.57941
59.64762
51.09050
58.98478
64.34459
74.53309


In [36]:
df.train.modified = df.train.copy %>%
    mutate(LotFrontageCalc = 
             ifelse(Neighborhood == 'BrkSide' & MSZoning == "RL", 
                  predicted,
                  LotFrontageCalc
             )
    )

head(df.train.modified %>% filter(Neighborhood == 'BrkSide' & MSZoning == "RL") %>% select(LotFrontageCalc))

LotFrontageCalc
58.02228
98.83280
77.94128
79.25051
58.02228
53.48631


In [37]:
df.train.modified2 = df.train.copy %>% rowwise() %>%
    mutate(LotFrontageCalc = 
             ifelse(Neighborhood == 'BrkSide' & MSZoning == "RL", 
                  predicted,
                  LotFrontageCalc
             )
    )

head(df.train.modified2 %>% filter(Neighborhood == 'BrkSide' & MSZoning == "RL") %>% select(LotFrontageCalc))

LotFrontageCalc
59.57941
59.57941
59.57941
59.57941
59.57941
59.57941
